### Import libraries

In [41]:
import pandas as pd
import numpy as np
from functools import partial
from functools import reduce
from datetime import datetime
import utils

Read file from local directory

In [20]:
with open('Mousotron.log', 'r') as log:
    logs = log.readlines()


In order to process the data from the log directly, i will consider the spaces in the document and identify how many are them for the column and for the data. Once i know these numbers i can effectively pull in the data.

- Date: 0 for column name, 3 spaces between the data. 
- days hour min sec: 3 for column name, 7 spaces between the data. (will use tuple notation for the next ones)
- Distance 1: (1, 1)
- Distance 2: (1, 1)
- Distance 3: (1, 1)
- keystrokes: (0,0)
- left: (0,0)
- right: (0,0)
- middle: (0,0)
- double: (0,0)
- wheel: (0,0)
- x1 btn: (1,0)
- x2 btn: (1,0)
- idle days hour min sec: (4,7)

In [113]:
from utils import spaces

cols = logs[0].split()
rows = [x.split() for x in logs[1:]]

def format_row(row, column_names=False, spaces=None):
    i_line = 0
    new_row = []
    spaces_idx = 0 if column_names else 1
    for col_spaces in spaces:
        slice_range = col_spaces[spaces_idx] + 1
        new_row.append(" ".join(row[i_line:slice_range + i_line]))
        i_line += slice_range
    return new_row

new_cols = format_row(cols, column_names=True, spaces=spaces)
df = pd.DataFrame(list(map(partial(format_row, column_names=False, spaces=spaces), rows)), columns = new_cols) # use partial to pass arguments to mapping function
df

,Date,days hour min sec,Distance 1 Distance 2 Distance 3,keystrokes,left,right,middle,double,wheel,x1 btn,x2 btn,idle days hour min sec
0,2020 abr. 07 23:32:52,0 days 13 hours 17 min 37 sec,1 km 43 m 3 cm,20590,3284,4758,15,2241,6683,0,0,0 days 7 hours 57 min 20 sec
1,2020 abr. 08 00:18:32,0 days 14 hours 3 min 17 sec,1 km 53 m 94 cm,20590,3363,4758,15,2248,6683,0,0,0 days 7 hours 57 min 33 sec
2,2020 abr. 09 00:00:12,0 days 14 hours 54 min 29 sec,0 km 663 m 26 cm,8535,2127,17,5,156,4854,0,0,0 days 0 hours 0 min 0 sec
3,2020 abr. 09 01:34:37,0 days 16 hours 28 min 54 sec,0 km 709 m 74 cm,9293,2314,17,5,167,5078,0,0,0 days 13 hours 25 min 58 sec
4,2020 abr. 10 14:05:30,0 days 2 hours 18 min 9 sec,1 km 316 m 7 cm,3935,3160,3849,1,1886,264,0,0,0 days 10 hours 24 min 27 sec
...,...,...,...,...,...,...,...,...,...,...,...,...
773,2022 nov. 01 15:55:54,26 days 16 hours 43 min 38 sec,12 km 43 m 61 cm,151297,35995,1933,137,3182,110598,0,0,19 days 19 hours 11 min 44 sec
774,2022 nov. 02 00:00:16,27 days 0 hours 46 min 13 sec,12 km 792 m 21 cm,158540,37063,1965,138,3280,114895,0,0,20 days 17 hours 46 min 28 sec
775,2022 nov. 03 00:00:57,28 days 0 hours 46 min 54 sec,13 km 891 m 48 cm,178014,40472,2235,142,3791,123208,0,0,21 days 10 hours 3 min 36 sec
776,2022 nov. 04 00:00:54,29 days 0 hours 46 min 51 sec,15 km 149 m 67 cm,187828,43436,2311,143,4052,133183,0,0,22 days 3 hours 43 min 55 sec


Dropping x1 and x2 btn rows because they don't bring in useful data, so we drop them

In [114]:
df[['x1 btn', 'x2 btn']].value_counts()

x1 btn  x2 btn
0       0         513
5       10        265
dtype: int64

In [115]:
df.drop(['x1 btn', 'x2 btn'], inplace=True, axis=1)

Renaming columns

In [119]:
df.rename({
    'days hour min sec': 'Active time',
    'Distance 1 Distance 2 Distance 3': 'Distance',
    'keystrokes': 'Keystrokes',
    'left': 'Left clicks',
    'right': 'Right clicks',
    'middle': 'Middle clicks',
    'double': 'Double clicks',
    'wheel': 'Wheel turn',
    'idle days hour min sec': 'Idle time'
}, inplace=True, axis=1)


In [120]:
df

,Date,Active time,Distance,Keystrokes,Left clicks,Right clicks,Middle clicks,Double clicks,Wheel turn,Idle time
0,2020 abr. 07 23:32:52,0 days 13 hours 17 min 37 sec,1 km 43 m 3 cm,20590,3284,4758,15,2241,6683,0 days 7 hours 57 min 20 sec
1,2020 abr. 08 00:18:32,0 days 14 hours 3 min 17 sec,1 km 53 m 94 cm,20590,3363,4758,15,2248,6683,0 days 7 hours 57 min 33 sec
2,2020 abr. 09 00:00:12,0 days 14 hours 54 min 29 sec,0 km 663 m 26 cm,8535,2127,17,5,156,4854,0 days 0 hours 0 min 0 sec
3,2020 abr. 09 01:34:37,0 days 16 hours 28 min 54 sec,0 km 709 m 74 cm,9293,2314,17,5,167,5078,0 days 13 hours 25 min 58 sec
4,2020 abr. 10 14:05:30,0 days 2 hours 18 min 9 sec,1 km 316 m 7 cm,3935,3160,3849,1,1886,264,0 days 10 hours 24 min 27 sec
...,...,...,...,...,...,...,...,...,...,...
773,2022 nov. 01 15:55:54,26 days 16 hours 43 min 38 sec,12 km 43 m 61 cm,151297,35995,1933,137,3182,110598,19 days 19 hours 11 min 44 sec
774,2022 nov. 02 00:00:16,27 days 0 hours 46 min 13 sec,12 km 792 m 21 cm,158540,37063,1965,138,3280,114895,20 days 17 hours 46 min 28 sec
775,2022 nov. 03 00:00:57,28 days 0 hours 46 min 54 sec,13 km 891 m 48 cm,178014,40472,2235,142,3791,123208,21 days 10 hours 3 min 36 sec
776,2022 nov. 04 00:00:54,29 days 0 hours 46 min 51 sec,15 km 149 m 67 cm,187828,43436,2311,143,4052,133183,22 days 3 hours 43 min 55 sec


In [88]:
df.to_csv('mousotron_data.csv', index=False)

In [30]:
df = pd.read_csv('mousotron_data.csv')
df

,Date,Active time,Distance,Keystrokes,Left clicks,Right clicks,Middle clicks,Double clicks,Wheel turn,Idle time
0,2020 abr. 07 23:32:52,0 days 13 hours 17 min 37 sec,1 km 43 m 3 cm,20590,3284,4758,15,2241,6683,0 days 7 hours 57 min 20 sec
1,2020 abr. 08 00:18:32,0 days 14 hours 3 min 17 sec,1 km 53 m 94 cm,20590,3363,4758,15,2248,6683,0 days 7 hours 57 min 33 sec
2,2020 abr. 09 00:00:12,0 days 14 hours 54 min 29 sec,0 km 663 m 26 cm,8535,2127,17,5,156,4854,0 days 0 hours 0 min 0 sec
3,2020 abr. 09 01:34:37,0 days 16 hours 28 min 54 sec,0 km 709 m 74 cm,9293,2314,17,5,167,5078,0 days 13 hours 25 min 58 sec
4,2020 abr. 10 14:05:30,0 days 2 hours 18 min 9 sec,1 km 316 m 7 cm,3935,3160,3849,1,1886,264,0 days 10 hours 24 min 27 sec
...,...,...,...,...,...,...,...,...,...,...
773,2022 nov. 01 15:55:54,26 days 16 hours 43 min 38 sec,12 km 43 m 61 cm,151297,35995,1933,137,3182,110598,19 days 19 hours 11 min 44 sec
774,2022 nov. 02 00:00:16,27 days 0 hours 46 min 13 sec,12 km 792 m 21 cm,158540,37063,1965,138,3280,114895,20 days 17 hours 46 min 28 sec
775,2022 nov. 03 00:00:57,28 days 0 hours 46 min 54 sec,13 km 891 m 48 cm,178014,40472,2235,142,3791,123208,21 days 10 hours 3 min 36 sec
776,2022 nov. 04 00:00:54,29 days 0 hours 46 min 51 sec,15 km 149 m 67 cm,187828,43436,2311,143,4052,133183,22 days 3 hours 43 min 55 sec


### Column information
All measurements except for date are registered relative to the last reset time. There are some resets made in this log. We shoud be able to determine them using some cumulative information that suddenly drops. 

- Date: the moment when this information was logged
- Active time: time the mouse or the keyboard have been active. i.e: moving, clicking, keystroking, etc...
- Distance: Distance traveled
- Keystrokes: keystrokes done to keyboard
- Left clicks: Left clicks on mouse
- Right clicks: Right clicks on mouse
- Middle clicks: Middle clicks on mouse
- Double clicks: Double clicks on mouse
- Wheel turn: Wheel turn on mouse
- Idle time: time the mouse or the keyboard have been non active.

On a first glance, we can see that **Date**, **Active time**, **Idle time** and **Distance** have an uncomfortable format to process.

### Data formatting

- **Date**: Convert to datetime object
- **Active time, Idle time**: compress information and count seconds. Convert to integer
- **Distance**: compress information and count centimeters. Convert to integer

The rest of the columns will be converted to integers.

### Formating Date as datetime

we're going to use datetime library for this.

First let's see an example of the data as presented

In [29]:
# sample date: 2022 nov. 03 00:00:57
df.iloc[5]['Date']

'2020 abr. 11 15:32:19'

We have to remove the dot and capitalize the month in order to properly format it

In [21]:
test = "2022 abr. 03 00:00:57"
test.replace(test.split(" ")[1], test.split(" ")[1].replace(".", "").capitalize())

'2022 Abr 03 00:00:57'

We still need to translate the month names to english

In [27]:
test = "2022 abr. 03 00:00:57"
test.replace(test.split(" ")[1], utils.month_es_to_en[test.split(" ")[1].replace(".", "").capitalize()])

'2022 Apr 03 00:00:57'

That looks more like it, so lets use that on all the Date column

In [26]:

# get the whole string and select only the month to replace without the dot and capitalized.
def format_date(date_string):
    date_string = date_string.replace(date_string.split(" ")[1], utils.month_es_to_en[date_string.split(" ")[1].replace(".", "").capitalize()])
    return datetime.strptime(date_string, '%Y %b %d %H:%M:%S')

df['Date'].apply(format_date)

0     2020-04-07 23:32:52
1     2020-04-08 00:18:32
2     2020-04-09 00:00:12
3     2020-04-09 01:34:37
4     2020-04-10 14:05:30
              ...        
773   2022-11-01 15:55:54
774   2022-11-02 00:00:16
775   2022-11-03 00:00:57
776   2022-11-04 00:00:54
777   2022-11-05 13:59:54
Name: Date, Length: 778, dtype: datetime64[ns]

perfect, moving on to Active and Idle time...

### Format Active time and Idle time

In [38]:
test = df.iloc[4]['Active time']
test

'0 days 2 hours 18 min 9 sec'

Some code to get just the numbers and get the total amount of seconds from each part

In [53]:
numbers = list(map(int, test.split(" ")[::2]))
mult = 1
for i in range(len(numbers)):
    if i == 0:
        mult = 24*60*60
    if i == 1:
        mult = 60*60
    if i == 2:
        mult = 60
    elif i == 3:
        mult = 1

    numbers[i] = numbers[i]*mult
numbers

[0, 7200, 1080, 9]

Now lets reduce that so we get the total seconds

In [55]:
reduce(lambda x, y: x+y, numbers)

8289

We can now apply this to the entire series and get a new column

In [56]:
def get_seconds(time):
    numbers = list(map(int, time.split(" ")[::2]))
    mult = 1
    for i in range(len(numbers)):
        if i == 0:
            mult = 24*60*60
        if i == 1:
            mult = 60*60
        if i == 2:
            mult = 60

        numbers[i] = numbers[i]*mult
    return reduce(lambda x, y: x+y, numbers)
        

df['Active time'].apply(get_seconds)

0        50040
1        51600
2        55380
3        62520
4         8820
        ...   
773    2308860
774    2336340
775    2425200
776    2511420
777    2648160
Name: Active time, Length: 778, dtype: int64

Same logic for Idle time

In [57]:
df['Idle time'].apply(get_seconds)

0        29820
1        30600
2            0
3        51780
4        39060
        ...   
773    1713300
774    1793640
775    1852740
776    1917480
777    1914420
Name: Idle time, Length: 778, dtype: int64

### Format distance column


In [58]:
test = df.iloc[4]['Distance']
test

'1 km 316 m 7 cm'


This procedure is very much like Active time, it's actually a variation for get_seconds function. Let's call it get_cents and replace the values for the multipliers.

In [62]:
def get_cents(time):
    numbers = list(map(int, time.split(" ")[::2]))
    mult = 1
    for i in range(len(numbers)):
        if i == 0:
            mult = 100*1000
        if i == 1:
            mult = 100
        elif i == 2:
            mult = 1

        numbers[i] = numbers[i]*mult
    return reduce(lambda x, y: x+y, numbers)
        

df['Distance'].apply(get_cents)

0       104303
1       105394
2        66326
3        70974
4       131607
        ...   
773    1204361
774    1279221
775    1389148
776    1514967
777    1544390
Name: Distance, Length: 778, dtype: int64

### Replace columns

In [65]:
df['Date'] = df['Date'].apply(format_date)
df['Distance'] = df['Distance'].apply(get_cents)
df['Active time'] = df['Active time'].apply(get_seconds)
df['Idle time'] = df['Idle time'].apply(get_seconds)


AttributeError: 'Timestamp' object has no attribute 'split'

In [68]:
df = df.astype({'Keystrokes': 'int64', 'Left clicks': 'int64', 'Right clicks': 'int64', 'Middle clicks': 'int64', 'Double clicks': 'int64', 'Wheel turn': 'int64'})

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           778 non-null    datetime64[ns]
 1   Active time    778 non-null    int64         
 2   Distance       778 non-null    int64         
 3   Keystrokes     778 non-null    int64         
 4   Left clicks    778 non-null    int64         
 5   Right clicks   778 non-null    int64         
 6   Middle clicks  778 non-null    int64         
 7   Double clicks  778 non-null    int64         
 8   Wheel turn     778 non-null    int64         
 9   Idle time      778 non-null    int64         
dtypes: datetime64[ns](1), int64(9)
memory usage: 60.9 KB
